# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("city_weather.csv")
file.dropna()
file.head()


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Tuktoyaktuk,40,CA,1610245021,84,69.4541,-133.0374,-4.00,13.80
1,1,Verāval,13,IN,1610245081,59,20.9000,70.3667,70.21,11.32
2,2,Rikitea,100,PF,1610245008,75,-23.1203,-134.9692,78.87,13.80
3,3,Tuatapere,89,NZ,1610244861,72,-46.1333,167.6833,64.00,21.99
4,4,Amberley,1,US,1610245081,86,39.2048,-84.4280,28.99,1.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
# Store 'Lat' and 'Lng' into  locations

cities = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)
max_humidity = humidity.max()

In [5]:
# Plot Heatmap
fig1 = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(cities, weights=humidity,
                               dissipating=False, max_intensity=max_humidity,
                               point_radius = 3)
fig1.add_layer(heat_layer)

#Display figure
fig1


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities to fit weather conditions. Drop any rows will null values

temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather_df=wind_file.loc[(wind_file['Cloudiness']==0)]

# Display results
ideal_weather_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,9,Maceió,0,BR,1610244813,88,-9.6658,-35.7353,75.2,4.61
33,33,Saint-Joseph,0,RE,1610245016,60,-21.3667,55.6167,75.2,5.75
65,65,Saint-Philippe,0,RE,1610245008,60,-21.3585,55.7679,75.2,5.75
71,71,Saint-Pierre,0,RE,1610245089,60,-21.3393,55.4781,75.2,5.75
78,78,Itaberaba,0,BR,1610244913,79,-12.5275,-40.3069,74.3,5.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a list variable for the hotels
hotel_df = ideal_weather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# Set up parameters dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

{
    "html_attributions": [],
    "next_page_token": "ATtYBwK_a6IGOow4NXX2zv2VHN0WZJaKfRAcDxZOLWcAxyext7q0IZtdaDkqHp5Al5k8cj03oWhjfOq-pqQiM4gGZXKOt8tcNDRmx1a9jgCwyrWptK3NlqFIEyeCSyNE-m_2m5ERPrM-2nRu35mCImZoBuPGDKvWN0Q9PaWGJgRHBMJ9xNMxVPzi8nglTpcFDl-C5hrU0k3t7eJIhLxWPbOpZHYLqNPJ1o5W0T7IV0eExj7nmzPSOucah0fgU1zppg_m1ZTXc2_6FqzPLn8F0JaR9SglAJ7fsr5fseMdxEhTL_j0eNjci2-vuhSbtOYFVJKiN4W9O6REUHTL_ZWIEULF7-SzcQo9dN0sLez2NVPvpoddCeTVA0rEnWSlO8jOO60KHmCrSjxYIIoQ-_ZQenhITmCcE7T-KCuQyHP0RigogYbSjD5g3dzH5QYFdnSB5QbV",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -9.5313577,
                    "lng": -35.6074351
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.529964020107277,
                        "lng": -35.60589542010727
                    },
                    "southwest": {
                        "lat": -9.532663

{
    "html_attributions": [],
    "next_page_token": "ATtYBwK9uLrT5TSFY4GmFZ-IFqwy0-dHQl1PXGaNq6_e6CNzVruKNtP4qa3tzzFkQ9sNWoB_rO9hWBMQfQfucce7QnKu5JmcpVtkW0FeK9yaj8aP_pNhl0riApwmjvS6FCtXiD4ugplfRgSXqKat9Tr5BeXMk-SAqGBqrHw6Y_NGTve4U9-1KCTmAiU4hwIDuFs2fNlNAFV1PaiHwcWhyaal0ClVNd2vm30NQYxf-0VHulYn6CblQFNXBeypmY1DZHvuH2PDhx_dkei_syUGBZxUxc-PLEhvUE57dJegz7nkdWJFZ2_o16rn1oacxM74ajHwR5FcAOZQUfYPIr6Idmti7B7hqnzFlfaMXUkGLCmfDJbIiF-ZuTi4B30BSmZPMKYVYzHna3C0FPbVdKmqMnIaK0pKEkAYY6CC3u1fIzU3azIaj5DsWNPr0urqp3wp8_Sw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.3668083,
                    "lng": 55.5506918
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.36545822010727,
                        "lng": 55.55204067989272
                    },
                    "southwest": {
                        "lat": -21.368157

{
    "html_attributions": [],
    "next_page_token": "ATtYBwImaUZ8UmZbG7gA3gARFZk-x2mIjufzpIEo5zgmP9N57A96wtNioFsY0T3zI5eLzoxo4BkMzkS8BeHDqa6mpwNBwPxWrsE_9_JM7b9WgOX5fnNOAlMOXYqegUFszDozER67F9zuSeJ04wi1Ih9ywF9Ztk4wxYGTR3L3oxzglUVjRcK8-7CFO2KlPMk6nfQYXQrtDVyMZSnMMILtnD3XwTQLYRSqNDC8Ycg5QILIErF8Ioa7re-tB7p0XiVGmD7g3ky8vcbnzL3F0TXKHkw8Eiuhg_otc081ypizB1kR8jNjgjlx3AJ7i3rcEqtDN_BiS7QOEYjbuDGRRCWNeb0ra0TDnzh-EoEUzAfPPUECb7jZCcadHTJH0dBVKPEwrnZQkqyLEl8NmikGYhmFwNbLUiQigIlb3OxrYUNMzaopgq38jkqF_5uebzypwTogsgPm",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.3668083,
                    "lng": 55.5506918
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.36545822010727,
                        "lng": 55.55204067989272
                    },
                    "southwest": {
                        "lat": -21.368157

{
    "html_attributions": [],
    "next_page_token": "ATtYBwL5765anNN2qdV-5PDUCHq0jDwfcMfobroHX7p9Y8KWUvvGu3vgSXiQnoBZWOVsIo-hwDEqDW86GToIK423HirlW-NjHbmipV_S3kSRui1bArnNrpXw2Si0sX_V4iejZaGlSXtwj2yakPSTDo1ZTPTnNYNY33Oc68py-Q5Dh-n9bkncmbnJf97_suCpB1JwthGZM3X1X5LdYnYpMh5sW-xoiDK3rKxMrmY0ockRGIrdR3uoq7FEwH8hO1PUkWNj0-BEI5CCfh8uqchMK30pipBZO19WMHeXKYjfcxcph0hyQfPCs23FnE_2wE9ycvY7pf_85Go76ek8ZSMBMYn1fkruFa8m3RfcUwG15YRK9y-nxEFg1UbVbJLdjJNTUrcaEkIO2bxKJDRGS6mQh3S0LtVZyvFfsWQiXPdS6S-2c1KTIbbo7fvV2cLWVg-YloHd",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.3668083,
                    "lng": 55.5506918
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.36545822010727,
                        "lng": 55.55204067989272
                    },
                    "southwest": {
                        "lat": -21.368157

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJd6c3Ng2k7lYkbQymmE_hQ7FDNYp8o_aubdyG1pz3-0VZS2wzvZQPEjjyskza3QdU7ikD4lSaSmrG865ptXT6PZbf6h0UVBZb9K4G3Ut8la2EaPZ_548dbDnjfk8CCsWtX67nlJY3AtgkCQVoVMz_ovnEf4FCNkSpaLjy4kq4JxuSrs5bL5Sqfu0b5Zu8E4x0ElBS2HoZoOaU7uU4F6YVuERrV_XxGFnwg2CDA5tTMLLVeMWbUBy1J4EflH5uKc0z5sUSK0fCkTZjyaycf6F3xw3mrMY9MtKyQRGDm246KOfnEMI4JtgPcug1SUQmvVQFL8kTvItDrZsZlKGc1oND1Nq1vJ-GURYO22PjGFlvmhE7FdWo-uh5ivweatjXVLWTskD2kOQOtS_wfREITEihn67_LqEA-w69c-_F8s6aGCBtp3jnr4hpxroCeQZTs",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -12.5294819,
                    "lng": -40.3027382
                },
                "viewport": {
                    "northeast": {
                        "lat": -12.52821502010728,
                        "lng": -40.30133517010728
                    },
                    "southwest": {
                        "lat": -12.5309

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJwrBu5OJo9deAWtn-ioH_ZWUwkwbONqfYabiJhbMUIk6k51vHpbxxmVxXXI1jCicAT19-VWaLtdb97gtTfV4ZVPnmtmoyCOOi94qGNeJf1DnTNNQMfCTOVPkiFnibygJR8gzaCAyl3GT9N-7Ls6gmZ7CvQ64FwU97GmSNTpfF0IIrCnsmbsi8qPoW4IXlxvYxwvUAclfMnC7UxCQihbOQ_HAOZlwCamZp9cX_W4tx_6h86ZZ0tSUxqUlA9V13A_upz-H6JCXUjHTS37S6sS6XoTZ9IpEgVYhQJPmdYL39_jCkh2GsTbQ6XZx0LnA0MXSidVwgjpm6uxkG_hl3Y6bQTtkDIOF88xVDqHATCTfepo5vJAPkp_qsi6sgnfohBP5-kRImWYi9hW9Tel_KrSq13PNJe9VhIStQ-ic_T5JkhcT9SAHyWne4YYTZR6E06",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -7.143678299999999,
                    "lng": -34.8134149
                },
                "viewport": {
                    "northeast": {
                        "lat": -7.142233170107278,
                        "lng": -34.81196872010728
                    },
                    "southwest": {
                        "lat": -

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.156932,
                    "lng": 37.2617452
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.15844132989272,
                        "lng": 37.26479365
                    },
                    "southwest": {
                        "lat": 25.15574167010728,
                        "lng": 37.26072904999999
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png",
            "name": "Royal Tours Permanent Camp",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/106088231879133419007\">F

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 9.515723699999999,
                    "lng": 31.6559863
                },
                "viewport": {
                    "northeast": {
                        "lat": 9.517404679892719,
                        "lng": 31.65753142989272
                    },
                    "southwest": {
                        "lat": 9.514705020107275,
                        "lng": 31.65483177010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Nile Palace Hotel",
            "place_id": "ChIJ6TMQvZQy_RYRSWqKBv8Fl-c",
            "plus_code": {
                "compound_code": "GM84+79 Malakal, South Sudan",
                "global_code": "6GXHGM84+79"
            },
            "ra

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIGSn4cRsPly5BgAxtYd8gdMKNo1vmVDcCeARCENiSW2zr60eKUnLRCHJDozfunjgR2V0XcaHSyu_oxw2VfBfQ7pVqgsM9j-vLgOCXoaV0_mJtef5U_cCNzx6G7pyb8E7D1z6t2GcN3WTq-xW7NrL7n8xX7-DOqKS1hLpor2NTgC4Wg1A7ybakuBdhNzpimZq7vbGS-yeBhUfxjFzu0BSGwghhUhUiuJHxhY99bKqZ5MyNaumFJ5KbJ6028RZYkDwooQlny1eOGLNVFPG3FI3wgs_o490KzyckjIuqvpag3CHFHoSRgNVoym4lQDBU63SurPA9CN-ahivQe5EAR72DF6KibKAFkX8zfFi0eikNgJHwbvwl1AQd23Y_5OOcgEVmdmxFFnGrfXepuZCUdDSfV8-ilo7UpBWwNVKQiIFJ0zTiWUTNdlivp1GO7SdYY",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.3668083,
                    "lng": 55.5506918
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.36545822010727,
                        "lng": 55.55204067989272
                    },
                    "southwest": {
                        "lat": -21.368157

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,9,Maceió,0,BR,1610244813,88,-9.6658,-35.7353,75.20,4.61,Salinas Maceió All Inclusive Resort
1,33,Saint-Joseph,0,RE,1610245016,60,-21.3667,55.6167,75.20,5.75,Palm Hotel & Spa
2,65,Saint-Philippe,0,RE,1610245008,60,-21.3585,55.7679,75.20,5.75,Palm Hotel & Spa
3,71,Saint-Pierre,0,RE,1610245089,60,-21.3393,55.4781,75.20,5.75,Palm Hotel & Spa
4,78,Itaberaba,0,BR,1610244913,79,-12.5275,-40.3069,74.30,5.50,Belavista Web Hotel
5,216,Conde,0,BR,1610244853,83,-7.2597,-34.9075,78.80,4.61,Val Atlantic Hotel
6,225,Umluj,0,SA,1610245108,55,25.0213,37.2685,73.71,6.46,Royal Tours Permanent Camp
7,258,Malakal,0,SS,1610245111,31,9.5334,31.6605,72.36,6.73,Nile Palace Hotel
8,267,Araouane,0,ML,1610245112,19,18.9048,-3.5265,72.41,9.69,
9,283,Tombouctou,0,ML,1610245114,20,20.0000,-3.0000,71.42,9.48,


In [8]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Show results of hotel_df

hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,9,Maceió,0,BR,1610244813,88,-9.6658,-35.7353,75.20,4.61,Salinas Maceió All Inclusive Resort
1,33,Saint-Joseph,0,RE,1610245016,60,-21.3667,55.6167,75.20,5.75,Palm Hotel & Spa
2,65,Saint-Philippe,0,RE,1610245008,60,-21.3585,55.7679,75.20,5.75,Palm Hotel & Spa
3,71,Saint-Pierre,0,RE,1610245089,60,-21.3393,55.4781,75.20,5.75,Palm Hotel & Spa
4,78,Itaberaba,0,BR,1610244913,79,-12.5275,-40.3069,74.30,5.50,Belavista Web Hotel
5,216,Conde,0,BR,1610244853,83,-7.2597,-34.9075,78.80,4.61,Val Atlantic Hotel
6,225,Umluj,0,SA,1610245108,55,25.0213,37.2685,73.71,6.46,Royal Tours Permanent Camp
7,258,Malakal,0,SS,1610245111,31,9.5334,31.6605,72.36,6.73,Nile Palace Hotel
8,267,Araouane,0,ML,1610245112,19,18.9048,-3.5265,72.41,9.69,
9,283,Tombouctou,0,ML,1610245114,20,20.0000,-3.0000,71.42,9.48,


In [10]:
# Plot markers on top of the heatmap.
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig2 = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=hotel_info
)


# Add layer
fig2.add_layer(heat_layer)
fig2.add_layer(hotel_layer)
# Display figure
fig2


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…